<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/Node_Classification_in_the_Amazon_Product_Graph/blob/master/Node_Classification_in_the_Amazon_Product_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Node Classification in the Amazon Product Graph**

## **Libraries**

In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-geometric ogb

In [ ]:
from ogb.nodeproppred import NodePropPredDataset
import torch
from torch.serialization import add_safe_globals
from torch_geometric.data.data import Data
from torch_geometric.data.data import DataEdgeAttr
from torch_geometric.data import Data
from ogb.nodeproppred import NodePropPredDataset

In [ ]:
add_safe_globals({'torch_geometric.data.data.Data': Data})
add_safe_globals({'torch_geometric.data.data.DataEdgeAttr': DataEdgeAttr})

## **Loading the ogbn-products Dataset**

In [ ]:
# We load the dataset.
dataset = NodePropPredDataset(name='ogbn-products')
graph, labels = dataset[0]  # The main object of the graph is of type Data
split_idx = dataset.get_idx_split()  # Indices train/val/test

print(graph)

EOFError: 